In [1]:
from flask import Flask, render_template, request, redirect, url_for
import json

In [2]:
import cv2

In [3]:
import tensorflow

In [6]:
import numpy as np 
import cv2
import os 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.metrics import Accuracy

from keras.callbacks import ModelCheckpoint
import json

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
import ngrok

In [4]:
# Defining constants
IMG_WIDTH = 128
IMG_HEIGHT = 128
NUM_CHANNELS = 3   # 3 channels - coloured images 
NUM_CLASSES = 2    # 2 classes - cats & dogs
BATCH_SIZE = 32
EPOCHS = 50

In [5]:
def load_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    return img

In [6]:
def preprocess_data(img_data):
    img_data = img_data.astype('float32')
    img_data /= 255.0
    return img_data

In [7]:
def predictions(img):
    img = np.expand_dims(img, axis=0)
    inputs = tf.cast(img, tf.float32)

    conv_output = model(inputs, training=False)

    #print(f'Conv_output: {conv_output}')
    predictions = model.predict(inputs)
    #print(f'Predictions: {predictions[0]}')
    target_class = tf.argmax(predictions[0])
    #print(f'Target Class: {target_class}')
    predicted_class = int((target_class))
    accu = predictions[:, target_class][0]
    #print(f'Accurate (in %): {round(accu*100, 2)}%')
    return predicted_class

---

In [8]:
# MODEL ARCHITECTURE:
model = Sequential()

# 1st Conv. Layer : 32 Filters of 3x3 size with i/p shape:(224,224,3) & Maximum pooling into size 2x2
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, NUM_CHANNELS) ))
model.add(MaxPool2D(pool_size=(2,2)))

# 2nd Conv. Layer : 64 Filters of 3x3 size & Max. pooling into 2x2 size
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# Flattening the o/p of conv layers to single vector
model.add(Flatten())

# Adding Dense layers (Can be added as required)
model.add(Dense(64, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax')) # final outpu layer containing two classes cats and dogs

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 64)                3

In [10]:
# COMPILING the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

---

## LOADING TRAINED MODEL WEIGHTS:

In [12]:
model.load_weights("model_chckpnt_woImanip.h5")

--- 
## Testing:

In [13]:
os.listdir('dogs_1')[2]

'1_002.jpg'

In [14]:
cat_img_path = os.path.join('cats_0', '0_044.jpg')
dog_img_path = os.path.join('dogs_1', '1_002.jpg')

In [15]:
cat_img = load_image(cat_img_path)
dog_img = load_image(dog_img_path)

In [16]:
cat_inp_data = preprocess_data(cat_img)
dog_inp_data = preprocess_data(dog_img)

In [17]:
# Predictions

In [18]:
predictions(cat_inp_data)

1/1 [==============================] - 8s 8s/step


0

In [19]:
predictions(dog_inp_data)

1/1 [==============================] - 0s 76ms/step


1

model working...

---

## Web scraping

In [3]:
# import selenium
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# import cv2
# from bs4 import BeautifulSoup
# from fake_useragent import UserAgent
# from webdriver_manager.chrome import ChromeDriverManager
# import tldextract
# import os

# ua = UserAgent()

# opt = webdriver.ChromeOptions()
# opt.add_argument('user-agent={}'.format(ua.random))

# def fetch_screenshot(url,base_url, counter, name, mobile_view=False):
#     driver = webdriver.Chrome(options=opt)
#     view = "desktop"
#     if mobile_view:
#         driver.set_window_size(width=390, height=844)
#         view = "mobile"
#     else:
#         driver.set_window_size(1920, 1080)
#         view = "desktop"
        
#     if url[:8] == 'https://':
#         driver.get(url)
#     else:
#         driver.get(base_url+url)
        
#     print(base_url)
#     print(url)
#     time.sleep(1.5)
#     # allow_cookies(driver)
#     screenShot = driver.save_screenshot(os.path.join(os.getcwd(), 'dataset1', "{}-ss-{}-{}.png".format(view,name, counter)))
#     # img = cv2.imread("screenshot.png")
#     print("\nSS:::::", screenShot)
#     driver.close()
    
#     return screenShot
#     # return img


# def get_all_paths(url):
#     driver = webdriver.Chrome(options=opt)
#     # driver = webdriver.Chrome()
#     driver.get(url)
#     time.sleep(3)
#         # allow_cookies(driver)
    
#     paths = []
    
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
#     for link in soup.find_all('a', href=True):
#         path = link.get('href')
#             # path = 'https://{}'.format(path)
#             # if path and not path.startswith('javascript:') and not path.startswith('#'):
#         if driver.find_element(By.XPATH,'//*[@href="{}"]'.format(path)):
#             # if driver.find_element_by_link_text(path):
#             # links = driver.find_elements_by_partial_link_text(path)
#             # if len(links) == 1:
#             if url in path or path[0] == '/':

#                 print(path)
#                 paths.append(path)

# #                paths.append(path)
#     driver.close()
#     return paths

# # def allow_cookies(driver):
# #     # accept_cookies = driver.find_element(By.XPATH, "//button[@class='iubenda-cs-accept-btn']")
# #     # accept_cookies.click()
# #     time.sleep(2)


# if __name__ == "__main__":
    
#     # url = "https://tradingview.com/"
#     # url = "https://www.bajajfinserv.in/"
#     url = 'https://www.muthootfincorp.com/'
#     # url = "https://www.frog.co/"
#     # url = "https://www.momentumdesignlab.com/"
#     base_url = url[:-1]
#     # name = 'FROG'
#     name = 'MOMENTUM'
    
#     paths = get_all_paths(url)
#     direc = os.getcwd()
#     i = 0
#     for path in paths:
#         screenShot = fetch_screenshot(path, base_url, i, name)
#         screenShot = fetch_screenshot(path, base_url, i,name, True)
#         # cv2.imwrite(os.path.join(os.getcwd(), 'TEMP idea on 11', 'test_ss_fetched', 'screenshot_{}-{}'.format(name, i)), screenShot)
#         i += 1

from selenium import webdriver
from selenium.webdriver.common.by import By
from PIL import Image
import io
import numpy as np


def fetch_test(url):
    opt = webdriver.ChromeOptions()

    # Initialize Selenium Driver
    driver = webdriver.Chrome(options=opt)

    # Open the URL
    driver.get(url)

    # Take screenshot and get the binary data
    screenshot = driver.get_screenshot_as_png()

    # Close the driver
    driver.quit()

    # Use BytesIO to create a file-like object in memory
    screenshot_stream = io.BytesIO(screenshot)

    # Open the stream as an image with PIL
    image = Image.open(screenshot_stream)

    # Convert image to NumPy array
    image_np = np.array(image)

    # image_list = image_np.tolist()

    return image_np


# Building Flask APP:

In [4]:
app = Flask(__name__)



@app.route('/')
def hello():
    print('http://127.0.0.1:5000/')
    return "server running..."


@app.route('/ping')
def pingPong():
    data = {'mg': "pong"}
    return data

@app.route('/predict', methods=['POST'])
def cat_dog_pred():
    data = json.loads(request.get_json())
    print(data, "....Image recieved...\n")
    image = data['image']
    image = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    
    img_data = preprocess_data(image)
    output = predictions(img_data)

    return json.dumps({'predicted_class' : output})

    
    
@app.route('/test', methods=['POST'])
def testing():
    data = json.loads(request.get_data())
    print(data, '\n data recieved... pinged \n')
    return json.dumps({'message' : 0, 'hey': 1})

@app.route('/ss', methods=['POST'])
def ss():
    byte_encode = request.get_data()
    print(byte_encode)
    url = byte_encode.decode('utf-8')
    print(url)
    data = fetch_test(url)
    print(list(data.shape))
    return json.dumps({
        "Rating" : 0.75
    })
    

In [ ]:
if __name__ == '__main__':
    # ngrok.connect(9000, authtoken_from_env=True)
    # print(tunnel.url())
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


b'https://www.bajajfinservmarkets.in/partners/bajaj-finance-limited.html'
https://www.bajajfinservmarkets.in/partners/bajaj-finance-limited.html


Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)
127.0.0.1 - - [21/Jul/2023 23:34:36] "POST /ss HTTP/1.1" 200 -


[884, 1295, 4]


127.0.0.1 - - [22/Jul/2023 00:34:38] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:34:54] "GET / HTTP/1.1" 200 -


http://127.0.0.1:5000/


127.0.0.1 - - [22/Jul/2023 00:35:05] "GET /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:24] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:25] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:26] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:26] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:26] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:26] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:27] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:27] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:27] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:27] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:27] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:28] "POST /test-it-up HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2023 00:36:28] "POST /test-it-up HTTP/1.1" 404 -
127.0.0